In [1]:
# import dependencies
import spotipy
import spotipy.util as util
import pandas as pd
import pprint

## Grab csv files dataframe

In [2]:
top_2017 = pd.read_csv("assets/data/top2017.csv", encoding='latin-1')
top_2018 = pd.read_csv("assets/data/top2018.csv", encoding='unicode_escape')
top_2019 = pd.read_csv("assets/data/top2019.csv", encoding='unicode_escape')

### create search list of each year

In [3]:
# initialize list to scrape
search_data = []

# pull tracks and artists
tracks = list(top_2019['Track.Name'])
artists = list(top_2019['Artist.Name'])
# add each list to search_data
for i in range(len(tracks)):
    search_data.append([tracks[i], artists[i]])



In [4]:
def search_by_track(track, artist):
    token = util.prompt_for_user_token(username, scope, client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET,redirect_uri=SPOTIPY_REDIRECT_URI)
    # create spotipy object
    sp = spotipy.Spotify(auth=token)
    # search by track name 
    result = sp.search(q=track, type='track')
    # save first 10 results
    first_ten_albums = result['tracks']['items']
    # loop through each album 
    for album in first_ten_albums:
        # extract artist of each album
        possible_artist = album['album']['artists'][0]['name']
        # see if album artist matches query artist
        if possible_artist == artist:
            # match found, add to song-ids
            value = [artist, track, result['tracks']['items'][0]['id']]
            return value
    

def search_by_artist(artist, track):
    ''' returns correct track by artist to search'''
    token = util.prompt_for_user_token(username, scope, client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET,redirect_uri=SPOTIPY_REDIRECT_URI)
    # create spotipy object
    sp = spotipy.Spotify(auth=token)
  
    # search by artist name
    result = sp.search(q=artist, type='artist')
    # grab artist id
    artist_id = result['artists']['items'][0]['id']
    # search artist top tracks
    top_tracks = sp.artist_top_tracks(artist_id)
    top_10 = top_tracks['tracks']
    # loop through top tracks, check to see if any match track query
    for song in top_10:
        possible_song = song['name']
        # matches enough, match found
        if track[:2] == possible_song[:2] or track[-3:] == possible_song[-3:]:
            track = possible_song
            return track


def grab_genre(artist):
    ''' returns a genre of artist'''
    token = util.prompt_for_user_token(username, scope, client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET,redirect_uri=SPOTIPY_REDIRECT_URI)
    # create spotipy object
    sp = spotipy.Spotify(auth=token)

    # search by artist name
    result = sp.search(q=artist, type='artist')
    # grab artist id
    artist_id = result['artists']['items'][0]['id']
    # search artist top tracks
    genre = sp.artist(artist_id)['genres']
    return genre
 

In [5]:
#https://open.spotify.com/user/1298997744?si=ijwp9NZOQoyjO9qOZliclw
SPOTIPY_CLIENT_ID='7655188a18144343b07e44965cc38310'
SPOTIPY_CLIENT_SECRET='2bec9b47853e4ea3a419418a211ec45d'
SPOTIPY_REDIRECT_URI='https://google.com/'
scope = 'user-library-read'
# Grab username
username = "1298997744"

token = util.prompt_for_user_token(username, scope, client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET,redirect_uri=SPOTIPY_REDIRECT_URI)

results = []

# Create our spotify object
for query in search_data:
    if token:
        # create track & artist search variables
        track = query[0]
        artist = query[1]
        genre = grab_genre(artist)

        # run search by track to find results
        value = search_by_track(track, artist)

        # if search_by_track returns none, use search_by_artist.
        # else append to results
        if value == None:
            correct_track = search_by_artist(artist, track)
            # run search_by_track again
            value = search_by_track(correct_track, artist)
            value.append(genre)
            results.append(value)
        else:
            value.append(genre)
            results.append(value)   
    else:
        print("Can't get token for", username)

In [6]:
token = util.prompt_for_user_token(username, scope, client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET,redirect_uri=SPOTIPY_REDIRECT_URI)

# create spotipy object
sp = spotipy.Spotify(auth=token)

top_2019_data = []
for each in results:
    features = sp.audio_features(each[2])
    data = features[0]
    data['name'] = each[1]
    data['artists'] = each[0]
    data['year'] = '2019'
    data['genre'] = each[3]
    top_2019_data.append(data)

In [19]:
top2019_df = pd.DataFrame.from_dict(top_2019_data)
top_2019_df = top2019_df.drop(columns=['type', 'uri','track_href','analysis_url'])
top_2019_df = top_2019_df[['id','name','artists','danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo','duration_ms','time_signature','year','genre']]
top_2019_df

,id,name,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,genre
0,0TK2YIli7K1leLovkQiNik,Señorita,Shawn Mendes,0.759,0.540,9,-6.039,0,0.0287,0.03700,0.000000,0.0945,0.750,116.947,190960,4,2019,"[canadian pop, pop, post-teen pop, viral pop]"
1,2ksOAxtIxY8yElEWw8RhgK,China,Anuel AA,0.786,0.808,7,-3.702,1,0.0882,0.08460,0.000289,0.0822,0.609,105.027,301714,4,2019,"[latin, reggaeton flow, trap latino]"
2,0Ryd8975WihbObpp5cPW1t,boyfriend (with Social House),Ariana Grande,0.400,0.795,10,-3.731,0,0.4610,0.11900,0.000000,0.1590,0.702,190.097,186107,4,2019,"[dance pop, pop, post-teen pop]"
3,4evmHXcjt3bTUHD1cvny97,Beautiful People (feat. Khalid),Ed Sheeran,0.640,0.648,5,-8.113,0,0.1870,0.12400,0.000000,0.0802,0.548,92.977,197867,4,2019,"[pop, uk pop]"
4,0t3ZvGKlmYmVsDzBJAXK8C,Goodbyes (Feat. Young Thug),Post Malone,0.548,0.653,5,-3.875,1,0.0810,0.45600,0.000000,0.1030,0.163,150.068,174853,4,2019,"[dfw rap, melodic rap, rap]"
5,3HVWdVOQ0ZA45FuZGSfvns,I Don't Care (with Justin Bieber),Ed Sheeran,0.798,0.675,6,-5.041,1,0.0442,0.09120,0.000000,0.0894,0.842,101.956,219947,4,2019,"[pop, uk pop]"
6,1lOe9qE0vR9zwWQAOk6CoO,Ransom,Lil Tecca,0.745,0.642,7,-6.257,0,0.2870,0.02040,0.000000,0.0658,0.226,179.974,131240,4,2019,[melodic rap]
7,6b2RcmUt1g9N9mQ3CbjX2Y,How Do You Sleep?,Sam Smith,0.477,0.682,1,-4.931,0,0.0925,0.15300,0.000000,0.0763,0.345,110.567,202205,5,2019,"[pop, uk pop]"
8,2YpeDb67231RjR0MgVLzsG,Old Town Road - Remix,Lil Nas X,0.878,0.619,6,-5.560,1,0.1020,0.05330,0.000000,0.1130,0.639,136.041,157067,4,2019,"[country rap, lgbtq+ hip hop, pop rap]"
9,2Fxmhks0bxGSBdJ92vM42m,bad guy,Billie Eilish,0.701,0.425,7,-10.965,1,0.3750,0.32800,0.130000,0.1000,0.562,135.128,194088,4,2019,"[electropop, pop]"


### finds genre of artist

In [22]:
def add_genre(dataframe):
    '''takes in a dataframe, adds a genre column by artist'''
    token = util.prompt_for_user_token(username, scope, client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET,redirect_uri=SPOTIPY_REDIRECT_URI)
    
    # grab artist column for Spotipy search
    artists = list(dataframe['artists'])  
   
    # initialize genre column
    genres = []

    for person in artists:
        # create spotipy object
        sp = spotipy.Spotify(auth=token)
        # search by artist name
        result = sp.search(q=person, type='artist')
        try:
            # grab artist id
            artist_id = result['artists']['items'][0]['id']
            # search artist top tracks
            genre = sp.artist(artist_id)['genres']
            genres.append(genre)
        except IndexError:
            i = dataframe.index[dataframe['artists'] == person]
            song_df = dataframe.loc[i, 'name']
            song_list = song_df.values
            song = song_list[0]
            result = sp.search(q=song, type='track')
            artist = result['tracks']['items'][0]['artists'][0]['name']
            result = sp.search(q=artist, type='artist')
            genre = result['artists']['items'][0]['genres']
            genres.append(genre)
                   
    dataframe['genre'] = genres
    
    return dataframe


In [23]:
top_2017_df = add_genre(top_2017)
top_2018_df = add_genre(top_2018)

In [24]:
top_2017_df

,id,name,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,genre
0,7qiZfU4dY1lWllzX7mPBI,Shape of You,Ed Sheeran,0.825,0.652,1,-3.183,0,0.0802,0.581000,0.000000,0.0931,0.9310,95.977,233713,4,2017,"[pop, uk pop]"
1,5CtI0qwDJkDQGwXD1H1cL,Despacito - Remix,Luis Fonsi,0.694,0.815,2,-4.328,1,0.1200,0.229000,0.000000,0.0924,0.8130,88.931,228827,4,2017,"[latin, latin pop, puerto rican pop, tropical]"
2,4aWmUDTfIPGksMNLV2rQP,Despacito (Featuring Daddy Yankee),Luis Fonsi,0.660,0.786,2,-4.757,1,0.1700,0.209000,0.000000,0.1120,0.8460,177.833,228200,4,2017,"[latin, latin pop, puerto rican pop, tropical]"
3,6RUKPb4LETWmmr3iAEQkt,Something Just Like This,The Chainsmokers,0.617,0.635,11,-6.769,0,0.0317,0.049800,0.000014,0.1640,0.4460,103.019,247160,4,2017,"[electropop, pop, tropical house]"
4,3DXncPQOG4VBw3QHh3S81,I'm the One,DJ Khaled,0.609,0.668,7,-4.284,1,0.0367,0.055200,0.000000,0.1670,0.8110,80.924,288600,4,2017,"[dance pop, hip hop, miami hip hop, pop, pop r..."
5,7KXjTSCq5nL1LoYtL7XAw,HUMBLE.,Kendrick Lamar,0.904,0.611,1,-6.842,0,0.0888,0.000259,0.000020,0.0976,0.4000,150.020,177000,4,2017,"[conscious hip hop, hip hop, rap, west coast rap]"
6,3eR23VReFzcdmS7TYCrhC,It Ain't Me (with Selena Gomez),Kygo,0.640,0.533,0,-6.596,1,0.0706,0.119000,0.000000,0.0864,0.5150,99.968,220781,4,2017,"[edm, pop, tropical house]"
7,3B54sVLJ402zGa6Xm4YGN,Unforgettable,French Montana,0.726,0.769,6,-5.043,1,0.1230,0.029300,0.010100,0.1040,0.7330,97.985,233902,4,2017,"[hip hop, pop rap, rap, southern hip hop, trap]"
8,0KKkJNfGyhkQ5aFogxQAP,That's What I Like,Bruno Mars,0.853,0.560,1,-4.961,1,0.0406,0.013000,0.000000,0.0944,0.8600,134.066,206693,4,2017,[pop]
9,3NdDpSvN911VPGivFlV5d,I DonÕt Wanna Live Forever (Fifty Shades Darker),ZAYN,0.735,0.451,0,-8.374,1,0.0585,0.063100,0.000013,0.3250,0.0862,117.973,245200,4,2017,"[dance pop, pop, post-teen pop, uk pop]"


In [25]:
def remove_unpopular_genres(dataframe):
    final_genres = ['trap', 'hip hop', 'reggaeton', 'edm', 'latin', 'tropical house', 'pop rap', 'rap', 'pop']
    
    for i, genres in dataframe['genre'].iteritems():
        index = 0
        while index < len(genres):
            if genres[index] in final_genres:
                index += 1
            else:
                del(genres[index])
                
    return dataframe

In [26]:
def add_genre_other(dataframe):
    ''' adds other as genre if empty '''
    for i, genres in dataframe['genre'].iteritems():
        if len(genres) == 0:
            genres.append('other')

    return dataframe

In [27]:
def del_mult_genres(dataframe):
    ''' picks main genre (choosing least populated genre)'''
    final_genres = ['trap', 'hip hop', 'reggaeton', 'edm', 'latin', 'tropical house', 'pop rap', 'rap', 'pop']
    for i, song_genres in dataframe['genre'].iteritems():
        if len(song_genres) > 1:
            index = 0
            while index < len(final_genres):
                if final_genres[index] in song_genres:
                    dataframe.loc[i,'genre'] = final_genres[index]
                    break
                else:
                    index += 1
        else:
            dataframe.loc[i,'genre'] = song_genres

    return dataframe

In [28]:
each_df = [top_2017_df, top_2018_df, top_2019_df]
for df in each_df:
    df = remove_unpopular_genres(df)
    df = add_genre_other(df)
    df = del_mult_genres(df)

,id,name,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,genre
0,7qiZfU4dY1lWllzX7mPBI,Shape of You,Ed Sheeran,0.825,0.652,1,-3.183,0,0.0802,0.581000,0.000000,0.0931,0.9310,95.977,233713,4,2017,pop
1,5CtI0qwDJkDQGwXD1H1cL,Despacito - Remix,Luis Fonsi,0.694,0.815,2,-4.328,1,0.1200,0.229000,0.000000,0.0924,0.8130,88.931,228827,4,2017,latin
2,4aWmUDTfIPGksMNLV2rQP,Despacito (Featuring Daddy Yankee),Luis Fonsi,0.660,0.786,2,-4.757,1,0.1700,0.209000,0.000000,0.1120,0.8460,177.833,228200,4,2017,latin
3,6RUKPb4LETWmmr3iAEQkt,Something Just Like This,The Chainsmokers,0.617,0.635,11,-6.769,0,0.0317,0.049800,0.000014,0.1640,0.4460,103.019,247160,4,2017,tropical house
4,3DXncPQOG4VBw3QHh3S81,I'm the One,DJ Khaled,0.609,0.668,7,-4.284,1,0.0367,0.055200,0.000000,0.1670,0.8110,80.924,288600,4,2017,trap
5,7KXjTSCq5nL1LoYtL7XAw,HUMBLE.,Kendrick Lamar,0.904,0.611,1,-6.842,0,0.0888,0.000259,0.000020,0.0976,0.4000,150.020,177000,4,2017,hip hop
6,3eR23VReFzcdmS7TYCrhC,It Ain't Me (with Selena Gomez),Kygo,0.640,0.533,0,-6.596,1,0.0706,0.119000,0.000000,0.0864,0.5150,99.968,220781,4,2017,edm
7,3B54sVLJ402zGa6Xm4YGN,Unforgettable,French Montana,0.726,0.769,6,-5.043,1,0.1230,0.029300,0.010100,0.1040,0.7330,97.985,233902,4,2017,trap
8,0KKkJNfGyhkQ5aFogxQAP,That's What I Like,Bruno Mars,0.853,0.560,1,-4.961,1,0.0406,0.013000,0.000000,0.0944,0.8600,134.066,206693,4,2017,pop
9,3NdDpSvN911VPGivFlV5d,I DonÕt Wanna Live Forever (Fifty Shades Darker),ZAYN,0.735,0.451,0,-8.374,1,0.0585,0.063100,0.000013,0.3250,0.0862,117.973,245200,4,2017,pop


In [30]:
def fix_id(track, artist):
    ''' searches by track, pulls 10 albums if artist in
        album matches artist param,
        RETURNS: [artist, track, song-id]'''
    token = util.prompt_for_user_token(username, scope, client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET,redirect_uri=SPOTIPY_REDIRECT_URI)

    # create spotipy object
    sp = spotipy.Spotify(auth=token)
    # search by track name
    result = sp.search(q=track, type='track')

    # save first 10 results
    first_ten_albums = result['tracks']['items']
    # loop through each album
    for album in first_ten_albums:
        # extract artist of each album
        possible_artist = album['album']['artists'][0]['name']
        # see if album artist matches query artist
        if possible_artist == artist:
            # match found, add to song-ids
            value = result['tracks']['items'][0]['id']
            return value

In [31]:
def fix_dataframe_values(dataframe):
    ''' corrects IDs , incorrect tracks '''
    for i, column in dataframe.iterrows():
        track = column['name']
        artist = column['artists']

        try:
            correct_id = fix_id(track,artist)
            if correct_id != None:
                dataframe.loc[i, 'id'] = correct_id
            if correct_id == None:
                token = util.prompt_for_user_token(username, scope, client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET,redirect_uri=SPOTIPY_REDIRECT_URI)
                sp = spotipy.Spotify(auth=token)
                # search by track name
                result = sp.search(q=track, type='track')
                correct_id = result['tracks']['items'][0]['id']
                dataframe.loc[i, 'id'] = correct_id

        except IndexError:
            correct_track = search_by_artist(artist, track)
            if correct_track != None:
                dataframe.loc[i, 'name'] = correct_track
                correct_id = fix_id(correct_track, artist)
                if correct_id != None:
                    dataframe.loc[i, 'id'] = correct_id
                else:
                    token = util.prompt_for_user_token(username, scope, client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET,redirect_uri=SPOTIPY_REDIRECT_URI)
                    sp = spotipy.Spotify(auth=token)
                    # search by track name
                    result = sp.search(q=correct_track, type='track')
                    correct_id = result['tracks']['items'][0]['id']
                    dataframe.loc[i, 'id'] = correct_id
            else:
                track = track.split()[0]
                correct_id = fix_id(track,artist)

    return dataframe

In [32]:
#### Run on Dataframes 2017 & 2018 ####
top_2017_df = fix_dataframe_values(top_2017_df)
top_2018_df = fix_dataframe_values(top_2018_df)
        

In [33]:
def find_popularity(track_id):
    ''' uses track id to find popularity value '''
    token = util.prompt_for_user_token(username, scope, client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET,redirect_uri=SPOTIPY_REDIRECT_URI)
    sp = spotipy.Spotify(auth=token)
    result = sp.track(track_id)
    popularity = result['popularity']

    return popularity

def add_popularity_to_df(dataframe):
    for i, column in dataframe.iterrows():
        track_id = column['id']
        #print("track id is: ", track_id)
        popularity = find_popularity(track_id)
        #print(popularity)
        dataframe.loc[i, 'popularity'] = popularity

    return dataframe

In [34]:
#### Run on our dataframes
for df in each_df:
    df = add_popularity_to_df(df)

In [42]:
top_2018_df


,id,name,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,genre,popularity
0,6DCZcSspjsKoFjzjrWoCdn,God's Plan,Drake,0.754,0.449,7,-9.211,1,0.1090,0.033200,0.000083,0.5520,0.3570,77.169,198973,4,2018,hip hop,86.0
1,3ee8Jmje8o58CHK66QrVC2,SAD!,XXXTENTACION,0.740,0.613,8,-4.880,1,0.1450,0.258000,0.003720,0.1230,0.4730,75.023,166606,4,2018,other,88.0
2,0e7ipj03S05BNilyu5bRzt,rockstar (feat. 21 Savage),Post Malone,0.587,0.535,5,-6.090,0,0.0898,0.117000,0.000066,0.1310,0.1400,159.847,218147,4,2018,rap,89.0
3,3swc6WTsr7rl9DqQKQA55C,Psycho (feat. Ty Dolla $ign),Post Malone,0.739,0.559,8,-8.011,1,0.1170,0.580000,0.000000,0.1120,0.4390,140.124,221440,4,2018,rap,84.0
4,2G7V7zsVDxg1yRsu7Ew9RJ,In My Feelings,Drake,0.835,0.626,1,-5.833,1,0.1250,0.058900,0.000060,0.3960,0.3500,91.030,217925,4,2018,hip hop,84.0
5,7dt6x5M1jzdTEt8oCbisTK,Better Now,Post Malone,0.680,0.563,10,-5.843,1,0.0454,0.354000,0.000000,0.1360,0.3740,145.028,231267,4,2018,rap,88.0
6,58q2HKrzhC3ozto2nDdN4z,I Like It,Cardi B,0.816,0.726,5,-3.998,0,0.1290,0.099000,0.000000,0.3720,0.6500,136.048,253390,4,2018,pop rap,85.0
7,7ef4DlsgrMEH11cDZd32M6,One Kiss (with Dua Lipa),Calvin Harris,0.791,0.862,9,-3.240,0,0.1100,0.037000,0.000022,0.0814,0.5920,123.994,214847,4,2018,edm,85.0
8,76cy1WJvNGJTj78UqeA5zr,IDGAF,Dua Lipa,0.836,0.544,7,-5.975,1,0.0943,0.040300,0.000000,0.0824,0.5100,97.028,217947,4,2018,pop,83.0
9,08bNPGLD8AhKpnnERrAc6G,FRIENDS,Marshmello,0.626,0.880,9,-2.384,0,0.0504,0.205000,0.000000,0.1280,0.5340,95.079,202621,4,2018,other,84.0


In [50]:
def grab_correct_title(dataframe):
    ''' uses id to get the correct title '''
    token = util.prompt_for_user_token(username, scope, client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET,redirect_uri=SPOTIPY_REDIRECT_URI)
    
    for i, column in dataframe.iterrows():
        sp = spotipy.Spotify(auth=token)
        track_id = column['id']
        result = sp.track(track_id)
        title = result['name']
        dataframe.loc[i, 'name'] = title
    
    
    return dataframe

In [51]:
grab_correct_title(top_2018_df)

,id,name,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,genre,popularity
0,6DCZcSspjsKoFjzjrWoCdn,God's Plan,Drake,0.754,0.449,7,-9.211,1,0.1090,0.033200,0.000083,0.5520,0.3570,77.169,198973,4,2018,hip hop,86.0
1,3ee8Jmje8o58CHK66QrVC2,SAD!,XXXTENTACION,0.740,0.613,8,-4.880,1,0.1450,0.258000,0.003720,0.1230,0.4730,75.023,166606,4,2018,other,88.0
2,0e7ipj03S05BNilyu5bRzt,rockstar (feat. 21 Savage),Post Malone,0.587,0.535,5,-6.090,0,0.0898,0.117000,0.000066,0.1310,0.1400,159.847,218147,4,2018,rap,89.0
3,3swc6WTsr7rl9DqQKQA55C,Psycho (feat. Ty Dolla $ign),Post Malone,0.739,0.559,8,-8.011,1,0.1170,0.580000,0.000000,0.1120,0.4390,140.124,221440,4,2018,rap,84.0
4,2G7V7zsVDxg1yRsu7Ew9RJ,In My Feelings,Drake,0.835,0.626,1,-5.833,1,0.1250,0.058900,0.000060,0.3960,0.3500,91.030,217925,4,2018,hip hop,84.0
5,7dt6x5M1jzdTEt8oCbisTK,Better Now,Post Malone,0.680,0.563,10,-5.843,1,0.0454,0.354000,0.000000,0.1360,0.3740,145.028,231267,4,2018,rap,88.0
6,58q2HKrzhC3ozto2nDdN4z,I Like It,Cardi B,0.816,0.726,5,-3.998,0,0.1290,0.099000,0.000000,0.3720,0.6500,136.048,253390,4,2018,pop rap,85.0
7,7ef4DlsgrMEH11cDZd32M6,One Kiss (with Dua Lipa),Calvin Harris,0.791,0.862,9,-3.240,0,0.1100,0.037000,0.000022,0.0814,0.5920,123.994,214847,4,2018,edm,85.0
8,76cy1WJvNGJTj78UqeA5zr,IDGAF,Dua Lipa,0.836,0.544,7,-5.975,1,0.0943,0.040300,0.000000,0.0824,0.5100,97.028,217947,4,2018,pop,83.0
9,08bNPGLD8AhKpnnERrAc6G,FRIENDS,Marshmello,0.626,0.880,9,-2.384,0,0.0504,0.205000,0.000000,0.1280,0.5340,95.079,202621,4,2018,other,84.0


In [ ]:
final_genres = ['trap', 'hip hop', 'reggaeton', 'edm', 'latin', 'tropical house', 'pop rap', 'rap', 'pop']

In [ ]:
def remove_unpopular_genres(dataframe):
    final_genres = ['trap', 'hip hop', 'reggaeton', 'edm', 'latin', 'tropical house', 'pop rap', 'rap', 'pop']
    
    for i, genres in dataframe['genre'].iteritems():
        index = 0
        while index < len(genres):
            if genres[index] in final_genres:
                index += 1
            else:
                del(genres[index])
            


In [ ]:
def add_genre_other(dataframe):
    for i, genres in dataframe['genre'].iteritems():
        if len(genres) == 0:
            genres.append('other')

In [ ]:
def del_mult_genres(dataframe):
    for i, song_genres in dataframe['genre'].iteritems():
        if len(song_genres) > 1:
            index = 0
            while index < len(final_genres):
                if final_genres[index] in song_genres:
                    dataframe.loc[i,'genre'] = final_genres[index]
                    break
                else:
                    index += 1
        else:
            dataframe.loc[i,'genre'] = song_genres

In [ ]:
#del_mult_genres(top_2017)
remove_unpopular_genres(top_2017)

In [ ]:
add_genre_other(top_2017)

In [ ]:
del_mult_genres(top_2017)

In [ ]:
top_2017

In [ ]:
token = util.prompt_for_user_token(username, scope, client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET,redirect_uri=SPOTIPY_REDIRECT_URI)

# create spotipy object
sp = spotipy.Spotify(auth=token)
# search by artist name
result = sp.track('0TK2YIli7K1leLovkQiNik')
result['popularity']

In [ ]:
top_2019_df

In [ ]:
token = util.prompt_for_user_token(username, scope, client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET,redirect_uri=SPOTIPY_REDIRECT_URI)

for i, track_id in top_2018['id'].iteritems():        
    # create spotipy object
    sp = spotipy.Spotify(auth=token)
    # search by artist name
    result = sp.track(track_id)
    print(result['popularity'])

In [ ]:
top_2017

In [ ]:
def fix_id(track, artist):
    ''' searches by track, pulls 10 albums
        if artist in album matches artist param,
        RETURNS: [artist, track, song-id]'''
    token = util.prompt_for_user_token(username, scope, client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET,redirect_uri=SPOTIPY_REDIRECT_URI)
    
    # create spotipy object
    sp = spotipy.Spotify(auth=token)
    # search by track name
    result = sp.search(q=track, type='track')

    # save first 10 results
    first_ten_albums = result['tracks']['items']
    # loop through each album
    for album in first_ten_albums:
        # extract artist of each album
        possible_artist = album['album']['artists'][0]['name']
        # see if album artist matches query artist
        if possible_artist == artist:
            # match found, add to song-ids
            value = result['tracks']['items'][0]['id']
            return value
    # possibly multiple artists, if so, return first id value
    value = result['tracks']['items'][0]['id']
    return value

In [ ]:
def search_by_artist2(artist, track):
    ''' returns correct track by artist to use in
        search_by_track'''
    token = util.prompt_for_user_token(username, scope, client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET,redirect_uri=SPOTIPY_REDIRECT_URI)
    # create spotipy object
    sp = spotipy.Spotify(auth=token)

    # search by artist name
    result = sp.search(q=artist, type='artist')
    # grab artist id
    artist_id = result['artists']['items'][0]['id']
    # search artist top tracks
    top_tracks = sp.artist_top_tracks(artist_id)
    top_10 = top_tracks['tracks']

    # loop through top tracks, check to see if any match track query
    for song in top_10:
        possible_song = song['name']
        print(possible_song)
        # matches enough, match found
        if track[:2] == possible_song[:2] or track[-3:] == possible_song[-3:]:
            track = possible_song
            return track

In [ ]:
token = util.prompt_for_user_token(username, scope, client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET,redirect_uri=SPOTIPY_REDIRECT_URI)

def fix_dataframe_values(dataframe):
    ''' corrects IDs , incorrect tracks '''
    for i, column in dataframe.iterrows():  
        track = column['name']
        artist = column['artists']
        try:
            correct_id = fix_id(track,artist)
            print('correct id', correct_id)
            if correct_id != None:
                dataframe.loc[i, 'id'] = correct_id
            if correct_id == None:
                sp = spotipy.Spotify(auth=token)
                # search by track name
                result = sp.search(q=track, type='track')
                correct_id = result['tracks']['items'][0]['id']
                print("nones correct id is now: ", correct_id)
                dataframe.loc[i, 'id'] = correct_id


        except IndexError:
            print('COULDNT FIND', track, ' by ', artist)
            correct_track = search_by_artist2(artist, track)
            print('track is now', correct_track)
            if correct_track != None:
                dataframe.loc[i, 'name'] = correct_track
                correct_id = fix_id(correct_track, artist)
                dataframe.loc[i, 'id'] = correct_id
            else:
                track = track.split()[0]
                correct_id = fix_id(track,artist)
                print("found justing", correct_id)
            print()
            print()

    #top_2017.loc[i, 'id']

In [ ]:
correct_track = "I Don’t Wanna Live Forever (Fifty Shades Darker)"
artist = "ZAYN"

checkid = fix_id(correct_track, artist)

token = util.prompt_for_user_token(username, scope, client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET,redirect_uri=SPOTIPY_REDIRECT_URI)
    
# create spotipy object
sp = spotipy.Spotify(auth=token)
# search by track name
result = sp.search(q=correct_track, type='track')
result['tracks']['items'][0]['id']

In [ ]:
top_2017

In [ ]:
sp = spotipy.Spotify(auth=token)
# search by track name
track = 'Ric Flair Drip (& Metro Boomin)'
result = sp.search(q=track, type='track')
artist = (result['tracks']['items'][0]['artists'][0]['name'])
correct_id = (result['tracks']['items'][0]['id'])
print(artist)
print(correct_id)

In [ ]:
token = util.prompt_for_user_token(username, scope, client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET,redirect_uri=SPOTIPY_REDIRECT_URI)
sp = spotipy.Spotify(auth=token)

artist = 'Ozuna'
result = sp.search(q=artist, type='artist', limit = 30)
result['artists']['items']


#print(result['tracks']['items'][16]['artists'])
# artist_id = result['artists']['items'][0]['id']
# top_tracks = sp.artist_top_tracks(artist_id)
# top_10 = top_tracks['tracks']
# for song in top_10:
#     print(song['name'])

In [ ]:
top_2017

In [ ]:
def find_popularity(track_id):
    token = util.prompt_for_user_token(username, scope, client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET,redirect_uri=SPOTIPY_REDIRECT_URI)
    sp = spotipy.Spotify(auth=token)
    result = sp.track(track_id)
    popularity = result['popularity']
    return popularity

In [ ]:
top_2017

In [ ]:
    
for i, column in top_2017.iterrows():  
        track_id = column['id']
        print("track id is: ", track_id)
        popularity = find_popularity(track_id)
        print(popularity)
        top_2017.loc[i, 'popularity'] = popularity

In [ ]:
top_2017

In [ ]:
find_popularity('6rPO02ozF3bM7NnOV4h6s2')

In [ ]:
token = util.prompt_for_user_token(username, scope, client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET,redirect_uri=SPOTIPY_REDIRECT_URI)
sp = spotipy.Spotify(auth=token)
result = sp.track('6rPO02ozF3bM7NnOV4h6s2')
result